<h1>Análise para identificação de perfis de clientes de uma rede de e-commerce</h1>
<p><b>Instituição:</b> USP ESALQ</p>
<p><b>Ano</b>: 2023</p>
<p><b>Autor:</b> João Paulo Ribeiro dos Santos</p>
<p><b>Localização Projeto: </b> <a href = 'https://github.com/Joao-TheCosmosIsInfinite/Cluster-Analysis' target = "_blank">Git Hub</a></p>

[1. Bibliotecas/ Libs](#1)<br>
[2. Carregar Dados](#2)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.1. Analise Pre-liminar](#21)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.2. Relacionamento das Bases](#22)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[2.3. Dicionário dos Dados](#23)<br>
[3. Informação dos dataframes](#3)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1. Dimensão](#31)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.2. Dados Generalistas](#31)<br>

%pip install nb-mermaid


%%html
<script src="//cdn.rawgit.com/bollwyvl/53e64cdafba38461943b/raw/0815758d591dfaf0f4918b388aed1bf11d82160d/mermaid.full.js"></script>
<style>
    .mermaid *{font-family: sans-serif; }
    .mermaid .node, .mermaid .cluster{
      fill: white !important;
      stroke: black !important;
      stroke-width: 1px !important;
    }
    .mermaid div{
      text-align: center;
    }
    .mermaid .label{
      color: black;
    }
</style>
<script>$(function(){
    // mermaid load a touch weirdly: try immediately, but try again later if it's not available
    var initMermaid = function(){
        return (window.mermaid && mermaid.init()) || setTimeout(initMermaid, 50);         
    }
    initMermaid();

    // for live editing, re-render only the text of the current cell
    window.IPython && $(IPython.events).on("rendered.MarkdownCell", function(evt, data){
        // this is using a hacked mermaid that accepts some nodes!
        mermaid.init(undefined, data.cell.element.find(".mermaid"));
    });
});</script>

<a id ='1'></a>
<h3>1. Bibliotecas/ Libs</h3>

In [1]:
## Bibliotecas para análise inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import sys
import os

## Utilidades
sys.path.append(os.path.abspath(os.path.join('..')))
from _functions import funcoes_data_viz

In [2]:
## Definir numero maximo de linhas para visualizar
pd.set_option('display.max_rows', 200)

## Definir numero maximo de linhas para visualizar
pd.set_option('display.max_columns', 50)

<a id ='2'></a>
<h3>2. Carregar Dados</h3>

In [3]:
## Localização dos dados
path = os.path.abspath(os.path.join('..'))

## Objetos/ DataFrames

# Cliente
df_customers = pd.read_csv(path + '/output/olist_customers_dataset.csv')
# Itens de Pedido
df_order_items = pd.read_csv(path + '/output/olist_order_items_dataset.csv')
# Pagamentos de Pedido
df_order_payments = pd.read_csv(path + '/output/olist_order_payments_dataset.csv')
# Pedido
df_orders = pd.read_csv(path + '/output/olist_orders_dataset.csv')
# Produtos
df_products = pd.read_csv(path + '/output/olist_products_dataset.csv')
# Vendedores
df_sellers = pd.read_csv(path + '/output/olist_sellers_dataset.csv')

<a id ='21'></a>
<h4>2.1. Relacionamento das bases</h4>

Relacionamento das tabelas/ datasets

<img src='../img/table_relation.png' width="1000" height="1000">

<a id ='22'></a>
<h4>2.2. Dicionário dos dados</h4>

<h3>Customer</h3>
<table>
    <tr><th>Campo</th><th>Descrição</th><th>Tipo de Variável</th></tr>
    <tr><td>customer_id</td><td>Identificador único do cliente</td><td>Categórica Nominal</td></tr>
    <tr><td>customer_unique_id</td><td>Identificador único de algum documento do cliente</td><td>Categórica Nominal</td></tr>
    <tr><td>customer_zip_code_prefix</td><td>Código postal</td><td>Categórica Nominal</td></tr>
    <tr><td>customer_city</td><td>Cidade onde reside o cliente</td><td>Categórica Nominal</td></tr>
    <tr><td>customer_state</td><td>Estado onde reside o cliente</td><td>Categórica Nominal</td></tr>
</table>


<h3>Order Items</h3>
<table>
    <tr><th>Campo</th><th>Descrição</th><th>Tipo de Variável</th></tr>
    <tr><td>order_id</td><td>Identificador único do pedido</td><td>Categórica Nominal</td></tr>
    <tr><td>order_item_id</td><td>Identificador númerico referente ao item no pedido</td><td>Categórica Nominal</td></tr>
    <tr><td>product_id</td><td>Identificador único do produto</td><td>Categórica Nominal</td></tr>
    <tr><td>seller_id</td><td>Identificador único do vendedor</td><td>Categórica Nominal</td></tr>
    <tr><td>shipping_limit_date</td><td>Data limite de entrega</td><td>Numérica Contínua</td></tr>
    <tr><td>price</td><td>Preço do item no pedido</td><td>Numérica Contínua</td></tr>
    <tr><td>freight_value</td><td>Valor do frete cobrado</td><td>Numérica Contínua</td></tr>
</table>


<h3>Order Payments</h3>
<table>
    <tr><th>Campo</th><th>Descrição</th><th>Tipo de Variável</th></tr>
    <tr><td>order_id</td><td>Identificador único do pedido</td><td>Categórica Nominal</td></tr>
    <tr><td>payment_sequential</td><td>Identificador sequencial único do pagamento</td><td>Categórica Nominal</td></tr>
    <tr><td>payment_type</td><td>Tipo de pagamento</td><td>Categórica Nominal</td></tr>
    <tr><td>payment_installments</td><td>Número de parcelas</td><td>Numérica Discreta</td></tr>
    <tr><td>payment_value</td><td>Valor total do pagamento</td><td>Numérica Contínua</td></tr>
</table>


<h3>Orders</h3>
<table>
    <tr><th>Campo</th><th>Descrição</th><th>Tipo de Variável</th></tr>
    <tr><td>customer_id</td><td>Identificador único do cliente</td><td>Categórica Nominal</td></tr>
    <tr><td>order_status</td><td>Status do pedido</td><td>Categórica Nominal</td></tr>
    <tr><td>order_purchase_timestamp</td><td>Data de quando o pedido foi feito</td><td>Numérica Contínua</td></tr>
    <tr><td>order_approved_at</td><td>Data de quando o pedido foi aprovado</td><td>Numérica Contínua</td></tr>
    <tr><td>order_delivered_carrier_date</td><td>Data de quando foi entregue a transportadora</td><td>Numérica Contínua</td></tr>
    <tr><td>order_delivered_customer_date</td><td>Data de quando foi entregue ao cliente</td><td>Numérica Contínua</td></tr>
    <tr><td>order_estimated_delivery_date</td><td>Data estimada para entrega</td><td>Numérica Contínua</td></tr>
</table>


<h3>Products</h3>
<table>
    <tr><th>Campo</th><th>Descrição</th><th>Tipo de Variável</th></tr>    
    <tr><td>product_id</td><td>Identificador único do produto</td><td>Categórica Nominal</td></tr>
    <tr><td>product_category_name</td><td>Categoria do produto</td><td>Categórica Nominal</td></tr>
    <tr><td>product_name_lenght</td><td>Número de caracteres referentes ao nome</td><td>Numérica Discreta</td></tr>
    <tr><td>product_description_lenght</td><td>Número de caracteres referentes a descrição</td><td>Numérica Discreta</td></tr>
    <tr><td>product_photos_qty</td><td>Quantidade de fotos</td><td>Numérica Discreta</td></tr>
    <tr><td>product_weight_g</td><td>Peso em gramas</td><td>Numérica Contínua</td></tr>
    <tr><td>product_length_cm</td><td>Comprimento em centímetros</td><td>Numérica Contínua</td></tr>
    <tr><td>product_height_cm</td><td>Altura em centímetros</td><td>Numérica Contínua</td></tr>
    <tr><td>product_width_cm</td><td>Largura em centímetros</td><td>Numérica Contínua</td></tr>
</table>


<h3>Sellers</h3>
<table>
    <tr><th>Campo</th><th>Descrição</th><th>Tipo de Variável</th></tr>    
    <tr><td>seller_id</td><td>Identificador único do vendedor</td><td>Categórica Nominal</td></tr>
    <tr><td>seller_zip_code_prefix</td><td>Código postal do vendedor</td><td>Categórica Nominal</td></tr>
    <tr><td>seller_city</td><td>Cidade onde o vendedor reside</td><td>Categórica Nominal</td></tr>
    <tr><td>seller_state</td><td>Estado onde o vendedor reside</td><td>Categórica Nominal</td></tr>    
</table>

<a id ='23'></a>
<h4>2.3. Analise Preliminar</h4>

In [4]:
## Cliente
df_customers.head(3)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP


In [5]:
## Item de Pedido
df_order_items.head(3)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.9,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.9,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.0,17.87


In [6]:
## Pagamento de Pedido
df_order_payments.head(3)

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71


In [7]:
## Pedido
df_orders.head(3)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00


In [8]:
## Produtos
df_products.head(3)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0


In [9]:
## Vendedores
df_sellers.head(3)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ


<a id = '3'></a>
<h3>3. Informação dos dataframes</h3>

<a id ='31'></a>
<h4>3.1. Dimensão</h4>

In [10]:
## Dimensão do dataframe
print('Dimensão do dataframe "df_cutomers":',df_customers.shape)
print('Dimensão do dataframe "df_order_items":', df_order_items.shape)
print('Dimensão do dataframe "df_order_payments":',df_order_payments.shape)
print('Dimensão do dataframe "df_orders":',df_orders.shape)
print('Dimensão do dataframe "df_products":',df_products.shape)
print('Dimensão do dataframe "df_sellers":',df_sellers.shape)

Dimensão do dataframe "df_cutomers": (99441, 5)
Dimensão do dataframe "df_order_items": (112650, 7)
Dimensão do dataframe "df_order_payments": (103886, 5)
Dimensão do dataframe "df_orders": (99441, 8)
Dimensão do dataframe "df_products": (32951, 9)
Dimensão do dataframe "df_sellers": (3095, 4)


<a id ='32'></a>
<h4>3.2. Dados Generalistas</h4>

In [11]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [12]:
df_order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [13]:
df_order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [14]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [15]:
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [16]:
df_sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB
